# Using rabbitmq-server Message Broker with Amazon AWS

### Installation

### check notes on http://kronosapiens.github.io/blog/2015/04/28/rabbitmq-aws.html

Install rabbitmq-server server first:

In [1]:
#!sudo apt-get install rabbitmq-server

check if the rabbitmq server is up and runnning: 

In [2]:
!sudo rabbitmqctl status

Status of node 'rabbit@ip-172-31-2-17' ...
[{pid,1968},
 {running_applications,[{rabbit,"RabbitMQ","3.5.7"},
                        {mnesia,"MNESIA  CXC 138 12","4.13.3"},
                        {xmerl,"XML parser","1.3.10"},
                        {os_mon,"CPO  CXC 138 46","2.4"},
                        {sasl,"SASL  CXC 138 11","2.7"},
                        {stdlib,"ERTS  CXC 138 10","2.8"},
                        {kernel,"ERTS  CXC 138 10","4.2"}]},
 {os,{unix,linux}},
 {erlang_version,"Erlang/OTP 18 [erts-7.3] [source] [64-bit] [smp:4:4] [async-threads:64] [kernel-poll:true]\n"},
 {memory,[{total,56678656},
          {connection_readers,0},
          {connection_writers,0},
          {connection_channels,0},
          {connection_other,6864},
          {queue_procs,37976},
          {queue_slave_procs,0},
          {plugins,0},
          {other_proc,13700024},
          {mnesia,68136},
          {mgmt_db,0},
          {msg_index,294784},
          {other_ets,857280},
        

RabbitMQ comes with a “guest” user out of the box, but this user is not configured to accept external requests. This would be fine if you were planning on sending and consuming messages locally, but we are not.

The easiest way to configure RabbitMQ to accept external requests is to create a new user. Go ahead and run the following. (Feel free to chahnge the myguest username and password to anytihng you like):

In [3]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

This will create a new user on the RabbitMQ server. We will use this username and password to configure the app-layer instance to send messages to Rabbit.

Let’s check to make sure the user was created correctly:

In [4]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


Pika is a pure-Python implementation of the AMQP 0-9-1 protocol that is used by RabbitMQ.

### For more details see  https://www.rabbitmq.com/tutorials/tutorial-one-python.html

In [5]:
#!pip install pika --upgrade

In [6]:
import pika

In [7]:
pika.__version__

'1.1.0'

#### Queues

A queue is the name for a post box which lives inside RabbitMQ. Although messages flow through RabbitMQ and your applications, they can only be stored inside a queue. A queue is only bound by the host's memory & disk limits, it's essentially a large message buffer. Many producers can send messages that go to one queue, and many consumers can try to receive data from one queue. 

### Example for sending message on Localhost 

In [8]:
# create a connection to 'localhost'
connection_local = pika.BlockingConnection(pika.ConnectionParameters('localhost'))


In [9]:
# create a channel on this connection:
channel_local = connection_local.channel()

In [10]:
# set up the queue
# Creating a queue using queue_declare is idempotent ‒ we can run the command as many times as we like, 
# and only one will be created.

channel_local.queue_declare(queue='hello_local')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=hello_local'])>"])>

<b> Before you run the next cell, run the 'recive_localhost' code as well. That program will wait for message

In [11]:
# send a message to this channel by specifiying a routing_key
# set the routing_key to the same as  the queue name!

channel_local.basic_publish(exchange='',
                      routing_key='hello_local',
                      body='Hello World 1!')
print(" [x] Sent 'Hello World 1!'")

 [x] Sent 'Hello World 1!'


<b> Check if the running 'recive_localhost' code recived the message!

In [12]:
connection_local.close()

### Here is another way with setting up credentials

In [13]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
parameters =  pika.ConnectionParameters('localhost', port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
channel.queue_declare(queue='hello_local')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=1', 'message_count=0', 'queue=hello_local'])>"])>

In [14]:
# set the routing key to be the queu name.
channel.basic_publish(exchange='',
                      routing_key='hello_local',
                      body='Hello World 2!')
print(" [x] Sent 'Hello World 2!'")

 [x] Sent 'Hello World 2!'


<b> check if the running 'receive_localhost' code received the message!

In [15]:
connection.close()

you can list the exisitng queues

In [16]:
!sudo rabbitmqctl list_queues

Listing queues ...
celery	0
hello_local	0
